In [21]:
import pickle
with open('pipelines.pickle', 'rb') as file:
    pipelines = pickle.load(file)

In [37]:
import pickle
with open('node_map2.pickle', 'rb') as file:
    node_map2 = pickle.load(file)

In [38]:
import networkx as nx
import re
from collections import defaultdict

def get_attr(str_node):
    try:
        attr,node = str_node.split('=')
    except:
        attr,node = '', str_node
    return(attr.strip(' )(,'),node.strip(' )(,'))

def get_link_node(substrings, level, count_subgraph):
    try:
        str_node = dict(dict(substrings).copy()[level]).copy()
        link_node = str_node[count_subgraph][-1][0]
    except:
        link_node = ''
    return(link_node)    

def add_nodes_from_pipeline(pipeline, graph):
    level = 0
    pipeline = re.sub('\s+', '', pipeline)
    substrings = defaultdict(lambda: defaultdict(list))
    substrings_for_edges = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    current_substring = ""
    count_subgraph = 0
    prev_c = pipeline[0]

    for c in pipeline:
        if c in ['(',',',')']:
            current_substring = current_substring.strip(' )(,')
            if current_substring:
                attr,node = get_attr(current_substring)
                # graph.add_node(f'{node}: {attr}') 
                graph.add_nodes_from([(f'{node}: {attr}', {'x': node_map2.get(f'{node}: {attr}',-1)})])
                link_node = get_link_node(substrings, level-1, count_subgraph)
                substrings[level][count_subgraph].append((f'{node}: {attr}', link_node))
                substrings_for_edges[count_subgraph][level][link_node].append(f'{node}: {attr}')
                current_substring = ""     

            if c == '(' and prev_c!='(':
                level += 1
            
            if c == '(' and prev_c==',':
                count_subgraph += 1    

            elif c == ')':
                level -= 1      
        else:
            current_substring += c
        prev_c = c

          
    if current_substring:
        current_substring.strip(' )(,')
        attr,node = get_attr(current_substring)
        # graph.add_node(f'{node}: {attr}')
        graph.add_nodes_from([(f'{node}: {attr}', {'x': node_map2.get(f'{node}: {attr}',-1)})])
        link_node = get_link_node(substrings, level-1, count_subgraph)
        substrings[level][count_subgraph].append((f'{node}: {attr}', link_node))
        substrings_for_edges[count_subgraph][level][link_node].append(f'{node}: {attr}')
            
    return substrings_for_edges, graph

def add_edges_from_pipeline(substrings, graph):
    for key,subgraph in substrings.items():
        for level_key in sorted(subgraph.keys(), reverse=True):
            for link_node, nodes in subgraph[level_key].items():
                if len(nodes)>2:
                    for a,b in zip(nodes[:-1], nodes[1:]):
                        if not ((a.endswith('numeric') and b.endswith('nominal')) or (b.endswith('numeric') and a.endswith('nominal'))):
                            graph.add_edge(a, b)
                        else:
                            graph.add_edge(a, link_node)
                            graph.add_edge(b, link_node)

                elif len(nodes)==2:
                    a,b = nodes
                    if not ((a.endswith('numeric') and b.endswith('nominal')) or (b.endswith('numeric') and a.endswith('nominal'))):
                        graph.add_edge(a, b)
                    else:
                        graph.add_edge(a, link_node)
                        graph.add_edge(b, link_node)

                    # graph.add_edge(*nodes)
                
                if link_node!='':
                    graph.add_edge(nodes[-1],link_node)
                elif not (key==0 and level_key==0 and link_node==''):
                    graph.add_edge(nodes[-1],substrings[0][0][''][0])
                    
    return graph                

In [39]:
def get_node_map(pipeline_graph):
    nodes = set()

    for graph in pipeline_graph:
        temp_nodes = set(graph.nodes)
        nodes = nodes.union(temp_nodes)

    
    nope_map = {node: idx for idx,node in enumerate(nodes)}

    return nope_map

In [40]:
def get_node_map2(pipeline_graph):
    
    nodes = set()

    for graph in pipeline_graph:
        if len(graph.edges) != 0:
            temp_nodes = set(graph.nodes)
            nodes = nodes.union(temp_nodes)

    nope_map = {node: idx for idx,node in enumerate(nodes)}

    return nope_map

In [41]:
from tqdm import tqdm
pipeline_graph = []
for pipeline in tqdm(pipelines):
    graph = nx.DiGraph()
    substrings, graph = add_nodes_from_pipeline(pipeline, graph)
    G = add_edges_from_pipeline(substrings, graph)
    pipeline_graph.append(G)

100%|██████████| 8098/8098 [00:00<00:00, 9428.91it/s] 


In [42]:
with open('pipeline_graph.pickle', 'wb') as file:
    pickle.dump(pipeline_graph, file)

In [35]:
pipeline_graph[0]['sklearn.compose._column_transformer.ColumnTransformer: columntransformer']

AtlasView({'sklearn.tree._classes.DecisionTreeClassifier: decisiontreeclassifier': {}})

In [34]:
pipeline_graph[0].nodes

NodeView(('sklearn.pipeline.Pipeline: ', 'sklearn.compose._column_transformer.ColumnTransformer: columntransformer', 'sklearn.impute._base.SimpleImputer: simpleimputer', 'sklearn.preprocessing._encoders.OneHotEncoder: onehotencoder', 'sklearn.tree._classes.DecisionTreeClassifier: decisiontreeclassifier'))

In [43]:
node_map = get_node_map(pipeline_graph)
pipeline_graph_rename = []
for graph in tqdm(pipeline_graph):
    G = nx.relabel_nodes(graph, node_map)
    pipeline_graph_rename.append(G)

100%|██████████| 8098/8098 [00:00<00:00, 16970.65it/s]


In [36]:
node_map2 = get_node_map2(pipeline_graph)
with open('node_map2.pickle', 'wb') as file:
    pickle.dump(node_map2, file)

In [32]:
# one_graph = nx.DiGraph()
# total_edges = 0

# for graph in tqdm(pipeline_graph):
#     one_graph = nx.disjoint_union(one_graph,graph)
#     total_edges += len(graph.edges)

100%|██████████| 8098/8098 [36:46<00:00,  3.67it/s]


In [12]:
with open('node_map.pickle', 'wb') as file:
    pickle.dump(node_map, file)

In [13]:
with open('pipeline_graph_rename.pickle', 'wb') as file:
    pickle.dump(pipeline_graph_rename, file)

In [44]:
pipeline_graph_rename[1].nodes

NodeView((1317,))

In [45]:
from torch_geometric.utils.convert import from_networkx
pyg_graph = from_networkx(pipeline_graph_rename[1])

In [46]:
pyg_graph.y = 0.34

In [47]:
pipeline_graph_rename[0].edges

OutEdgeView([(1258, 1145), (961, 1071), (1071, 1258), (1145, 1492)])

In [48]:
pipeline_graph_rename[0].nodes

NodeView((1492, 1258, 961, 1071, 1145))

In [49]:
dir(pyg_graph)

['__call__',
 '__cat_dim__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__inc__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_all_edges_to_layout',
 '_edge_attr_cls',
 '_edge_to_layout',
 '_get_edge_index',
 '_get_tensor',
 '_get_tensor_size',
 '_multi_get_tensor',
 '_put_edge_index',
 '_put_tensor',
 '_remove_tensor',
 '_store',
 '_tensor_attr_cls',
 '_to_type',
 'apply',
 'apply_',
 'batch',
 'clone',
 'coalesce',
 'contains_isolated_nodes',
 'contains_self_loops',
 'contiguous',
 'coo',
 'cpu',
 'csc',
 'csr',
 'cuda',
 'debug',
 'de

In [50]:
pyg_graph.x

tensor([-1])